In [2]:
import os.path as osp
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

import torch
from torchvision.datasets import CocoDetection

from detectron2.engine.defaults import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

from detectron2_timm.config import get_cfg

In [6]:
!apt-get update && apt-get install -y wget

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.19.4-1ubuntu2.2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


#### Download the pre-trained model from https://github.com/facebookresearch/xcit/tree/master/detection

In [25]:
!if [ ! -f ../logs/maskrcnn_xcit_medium_24_p8.pth ]; then wget https://dl.fbaipublicfiles.com/xcit/coco/maskrcnn_xcit_medium_24_p8.pth -P ../logs; else echo "Model is already downloaded..."; fi

Model is already downloaded...


### Convert the model keys to match the keys expected by `detectron2_timm`

In [31]:
!python ../tools/weight_converter.py --name mrcnn_xcit_medium_24_p8 --weights ../logs/maskrcnn_xcit_medium_24_p8.pth --output_dir ../logs/pretrained/

Done remapping keys for model:  ../logs/maskrcnn_xcit_medium_24_p8.pth
Done and saved to: ../logs/pretrained/mrcnn_xcit_medium_24_p8.pth


### Run the evaluation on Coco

In [35]:
!MKL_SERVICE_FORCE_INTEL=1 python ../tools/train_net.py --config-file ../config/xcit/xcit_medium_24_p8_224_fpn.yaml --num-gpus 1 --eval-only MODEL.WEIGHTS ../logs/pretrained/mrcnn_xcit_medium_24_p8.pth

Error: mkl-service + Intel(R) MKL: MKL_THREADING_LAYER=INTEL is incompatible with libgomp.so.1 library.
	Try to import numpy first or set the threading layer accordingly. Set MKL_SERVICE_FORCE_INTEL to force it.
Command Line Args: Namespace(config_file='../config/xcit/xcit_medium_24_p8_224_fpn.yaml', dist_url='tcp://127.0.0.1:49152', eval_only=True, machine_rank=0, num_gpus=1, num_machines=1, opts=['MODEL.WEIGHTS', '../logs/pretrained/mrcnn_xcit_medium_24_p8.pth'], resume=False)
[08/07 07:05:53 detectron2]: Rank of current process: 0. World size: 1
[08/07 07:05:53 detectron2]: Environment info:
----------------------  ---------------------------------------------------------
sys.platform            linux
Python                  3.8.8 (default, Feb 24 2021, 21:46:12) [GCC 7.3.0]
numpy                   1.19.2
detectron2              0.4.1 @/workspace/research/detectron2/detectron2
Compiler                GCC 7.5
CUDA compiler           not available
DETECTRON2_ENV_MODULE   <not set>
PyT

In [34]:
!nvidia-smi

Sat Aug  7 07:03:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 30%   31C    P8    20W / 320W |    926MiB / 10015MiB |     13%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------